<a href="https://colab.research.google.com/github/UPstartDeveloper/Problem_Solving_Practice/blob/master/practice_fall_2021/Property_revenue_across_cities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# InterviewQs
## Property revenue across cities


Hi, 
Suppose you work for Airbnb as an analyst. A team has come to you asking which cities generate the highest revenue for the company in 2017. Using the schemas below, write a SQL query to answer this question.

    
You have a table with property location information and another table with stay information. The schema of the tables are below:

    
Table: `property_location_info`
    
Column Name	Data Type	Description
property_id	integer	ID of the property location
country	string	country code of the property location
city_name	string	name of city **(note there can be multiple cities with the same name)**

subregion_name	string	provence, state, or subregion name
address	string	address of property location

Table: `stays_info`

Column Name	Data Type	Description
guest_id	integer	ID of guest
property_id	integer	ID of the property location
host_id	integer	ID of the host managing the property
revenue	integer	cost of stay for guest in USD
date_start	string	start day of stay, format is "YYYY-mm-dd"
date_end	string	end day of stay, format is "YYYY-mm-dd"
stay_length	integer	number of days for the stay
airbnb_revenue	integer	revenue that Airbnb collected on stay

In [21]:
import pandas as pd
import datetime as dt

In [22]:
# just testing out how dt values work
test_date = dt.date(2019, 5, 2)

In [23]:
str(test_date)

'2019-05-02'

In [24]:
# making up date values
dates = [
  dt.date(2019, 5, 2),
  dt.date(2017, 5, 2),
  dt.date(2017, 5, 3),
  dt.date(2017, 5, 4),
]

In [25]:
properties_df = pd.DataFrame({
    'property_ID': [10, 20, 30, 40, 50],  # int: ID of the property location
    'city_name': ['Baltimore', 'Denver', 'LA', 'NYC', 'SF'],  # str: city name,
    'subregion_name': ['MD', 'CO', 'CA', 'NY', 'CA']  # str: provence, state, or subregion name
})

In [26]:
properties_df.head()

,property_ID,city_name,subregion_name
0,10,Baltimore,MD
1,20,Denver,CO
2,30,LA,CA
3,40,NYC,NY
4,50,SF,CA


In [27]:
stays_df = pd.DataFrame({
    'property_ID': [10, 20, 10, 50],  # int: ID of the property location
    'date_end': [str(d) for d in dates], # str: end day of stay, format is "YYYY-mm-dd"
    'airbnb_revenue': [45, 59, 64, 1000]  # int: revenue that Airbnb collected on stay
})

In [28]:
stays_df.head()

,property_ID,date_end,airbnb_revenue
0,10,2019-05-02,45
1,20,2017-05-02,59
2,10,2017-05-03,64
3,50,2017-05-04,1000


In [29]:
# s = stays_df['date_end'].astype(str)
# s.str.startswith('2017')
stays_df[stays_df['date_end'].str.contains("2017")]

,property_ID,date_end,airbnb_revenue
1,20,2017-05-02,59
2,10,2017-05-03,64
3,50,2017-05-04,1000


In [90]:
def sort_cities_by_revenue(properties_df, stays_df, year='2017'):
  # A: get all the stays per property in 2017
  stays_2017 = stays_df[stays_df['date_end'].str.startswith("2017")]
  # B: aggregate the total revenue AirBnb got from the properties
  property_sums = stays_2017.groupby('property_ID', as_index=False).sum()
  # C: find out the unique cities
  unique_cities = properties_df.drop_duplicates(['city_name', 'subregion_name'])
  unique_cities['revenue_2017'] = 0
  # D: aggregate its total revenue, based on each unqiue property
  uc, ps = unique_cities, property_sums
  for prop_id in property_sums['property_ID'].values:
    city_revenue = ps[ps['property_ID'] == prop_id]['airbnb_revenue']
    uc.loc[uc['property_ID'] == prop_id, 'revenue_2017'] = int(city_revenue)
  # E: sort the cities from greatest to least
  return uc.sort_values('revenue_2017', ascending=0)['city_name'].values

In [91]:
sort_cities_by_revenue(properties_df, stays_df)

array(['SF', 'Baltimore', 'Denver', 'LA', 'NYC'], dtype=object)